<a href="https://colab.research.google.com/github/LiZhen97/Deep-Neural-Networks/blob/main/assignment_ZhenLi_class5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 5 Assignment: Predicting Home Prices**

**Student Name: Your Name**

# Assignment Instructions

This assignment asks you to train a neural network to predict housing prices.  I provide you with two different datasets.  You will use one data set, which includes house prices for training.  A second dataset, which does not include house prices, will be used for prediction and be submitted for evaluation.  I also give you a third dataset that contains median income for zip codes that must be joined to both the training/test datasets to provide an additional input value.  You must use the median income with your inputs for extra predictive power.

You can find all of the needed CSV files here:

* [House Prices - Training](https://data.heatonresearch.com/data/t81-558/datasets/houses_train.csv)
* [House Prices - Submit](https://data.heatonresearch.com/data/t81-558/datasets/houses_test.csv)
* [Median Income by Zipcode](https://data.heatonresearch.com/data/t81-558/datasets/zips.csv)

The median income by zipcode provides an additional feature, median income, that you should use in your predictions.  To complete this assignment perform the following steps:

* Load the housing prices training data.
* Join the median income by zipcode to the training data so that you gain the median income.
* Train a model to predict house price when given the following inputs: 'bedrooms', 'bathrooms', 'garage', 'land', 'sqft', 'median_income'.
* Load the housing prices test data.  This data does not contain the house price, you must predict this.
* Join the median income by zipcode to the test/submit data to gain the median income.
* Predict prices for the test/submit data.  
* Create a submission dataset that contains the house id (from the test/submit data) and the predicted price for that house.  Include no other fields.
* Submit this dataset and see how close you are to the actual values.

The RMSE should be less than 10K, which means that you are predicting within +/- $10,000 the actual price is sufficient to complete the assignment.  You may also wish to see if you can get your prediction even more accurate.

A few notes/suggestions on this assignment:

* You will see really high loss rates, due to the face that the target value is the price of a house (which is a large number)
* You might want to add a RMSE metric, which is the error in the same units target value. 
* I was able to obtain an RMSE of 86; yet still had a fairly high loss of 25920990.
* To add RMSE as a metric:
```
model.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer=opt)
```
# Google CoLab Instructions
If you are using Google CoLab, it will be necessary to mount your GDrive to send your notebook during the submit process. Running the following code will map your GDrive to /content/drive.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [2]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #5 Sample Code

The following code provides a starting point for this assignment.

In [13]:
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [52]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import zscore

# This is your student key that I emailed to you at the beginnning of the semester.
key = "eaT4qNtOTV4YNWzOIPMr83juoEsVZAkH7gPcxQ4P"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/My Drive/Colab Notebooks/assignment_ZhenLi_class5.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class5.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class5.ipynb'  # Mac/Linux

# Begin assignment
#df_houses_train = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/houses_train.csv")
#df_houses_submit = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/houses_test.csv")
#df_zips = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/zips.csv")


#submit(source_file=file,data=[df_submit],key=key,no=5)

In [ ]:
print(df_houses_train)
print(df_zips)

In [43]:
df_houses_train = pd.merge(df_houses_train, df_zips, on='zip')
df_houses_submit = pd.merge(df_houses_submit, df_zips, on='zip')

In [ ]:
df_houses_train

In [45]:
x = df_houses_train.drop(['price','id','zip'], axis=1)
x = x.values
y = df_houses_train['price'].values
#split dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=11)

In [15]:
x_train

array([[7.38500e+03, 6.00210e+04, 7.00000e+00, ..., 4.07200e+03,
        1.25572e+05, 1.25572e+05],
       [5.61400e+03, 6.00270e+04, 5.00000e+00, ..., 2.88800e+03,
        4.19118e+05, 4.19118e+05],
       [7.97800e+03, 6.00170e+04, 6.00000e+00, ..., 3.39600e+03,
        4.09450e+04, 4.09450e+04],
       ...,
       [3.66600e+03, 6.00050e+04, 9.00000e+00, ..., 5.88400e+03,
        4.92380e+05, 4.92380e+05],
       [9.11000e+03, 6.00330e+04, 8.00000e+00, ..., 4.97400e+03,
        4.26830e+05, 4.26830e+05],
       [6.99200e+03, 6.00090e+04, 4.00000e+00, ..., 2.45000e+03,
        6.74170e+04, 6.74170e+04]])

In [46]:
#Train a model to predict house price when given the following inputs: 'bedrooms', 'bathrooms', 'garage', 'land', 'sqft', 'median_income'.
model = Sequential()
model.add(Dense(20, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(11, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto', 
                        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

pred = model.predict(x_test)
# Measure MSE error.  
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))


Epoch 1/1000
282/282 - 2s - loss: 286231035904.0000 - val_loss: 175027273728.0000 - 2s/epoch - 6ms/step
Epoch 2/1000
282/282 - 1s - loss: 176645947392.0000 - val_loss: 172018745344.0000 - 522ms/epoch - 2ms/step
Epoch 3/1000
282/282 - 1s - loss: 173325811712.0000 - val_loss: 168359870464.0000 - 541ms/epoch - 2ms/step
Epoch 4/1000
282/282 - 1s - loss: 168986902528.0000 - val_loss: 163758931968.0000 - 515ms/epoch - 2ms/step
Epoch 5/1000
282/282 - 1s - loss: 163155181568.0000 - val_loss: 156476719104.0000 - 528ms/epoch - 2ms/step
Epoch 6/1000
282/282 - 0s - loss: 153564069888.0000 - val_loss: 145313300480.0000 - 497ms/epoch - 2ms/step
Epoch 7/1000
282/282 - 1s - loss: 134777806848.0000 - val_loss: 118142107648.0000 - 533ms/epoch - 2ms/step
Epoch 8/1000
282/282 - 1s - loss: 98543132672.0000 - val_loss: 71826259968.0000 - 510ms/epoch - 2ms/step
Epoch 9/1000
282/282 - 1s - loss: 49587519488.0000 - val_loss: 25910382592.0000 - 536ms/epoch - 2ms/step
Epoch 10/1000
282/282 - 1s - loss: 145521950

In [54]:
x2 = df_houses_submit.drop(['id','zip'], axis=1)
x2 = x2.values
pred_price = model.predict(x2)

63/63 [==============================] - 0s 1ms/step


In [55]:
df_Sb = pd.DataFrame(pred_price, columns=['price'])
df_Sb['id'] = df_houses_submit['id']

In [56]:
df_Sb

,price,id
0,1.103997e+06,10001
1,3.804884e+05,10014
2,2.732141e+05,10057
3,9.383687e+05,10142
4,2.727377e+05,10205
...,...,...
1995,1.009308e+06,11665
1996,7.975517e+05,11800
1997,4.981129e+05,11819
1998,3.618459e+05,11847


In [57]:
submit(source_file=file,data=[df_Sb],key=key,no=5)

Success: Submitted Assignment 5 for li.zhen1:
You have submitted this assignment 2 times. (this is fine)
Note: The mean difference 1261.6854500001064 for column 'price' is acceptable and is less than the maximum allowed value of '10000' for this assignment.
No warnings or errors (only notes), you will probably do well, but no guarantee. :-)
